In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math, random

In [2]:
def rescaleFrame(frame, scale=0.75):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)

    dimensions = (width,height)

    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [3]:
img = cv.imread("./unsplash.jpg", 0)
h, w = img.shape

factor = 0

if h >= 1920 or h >= 1080 or w >= 1920 or w >= 1080:
    factor = (max(h, w) // 1920)
    factor /= (10 * len(str(factor)))
    
img = rescaleFrame(img, factor)

In [4]:
cv.imshow("Unsplash Original", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
h, w = img.shape
for i in range (h):
    for j in range(w):
        if img[i][j] > 127:
            img[i][j] = 255
        else:
            img[i][j] = 0
            
cv.imshow("Unsplash Thresholded", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
def cropping(image):
    h, w = image.shape
    h = h//8*8
    w = w//8*8
    return image[0:h, 0:w]

In [7]:
img = cropping(img)
cv.imshow("Unsplash 8 Divisible", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
h, w= img.shape
sub_height, sub_width = (h//4, w//2)

In [9]:
def divide_and_shuffle(input_image, sub_height, sub_width, shuffle=False):
    parts_list = list()
    for x in range(4):
        parts_list.append(input_image[(x)*sub_height:(x+1)*sub_height, 0:sub_width])
        parts_list.append(input_image[(x)*sub_height:(x+1)*sub_height, sub_width:])
    
    return (random.sample(parts_list, len(parts_list)) if shuffle == True else parts_list)


parts_list = divide_and_shuffle(img, sub_height, sub_width, True)

In [10]:
def combine_shuffled_image(parts_list):
    for x in range(0, 8, 2):
        img_new = cv.hconcat([parts_list[x], parts_list[x+1]])
        parts_list.append(img_new)
    
    for x in range(8):
        parts_list.pop(0)
    
    for x in range(0, 4, 2):
        img_new = cv.vconcat([parts_list[x], parts_list[x+1]])
        parts_list.append(img_new)
        
    for x in range(4):
        parts_list.pop(0)
        
    return_img = cv.vconcat([parts_list[0], parts_list[1]])
    parts_list.clear()
    return return_img

img_new = combine_shuffled_image(parts_list)

cv.imshow("Unsplash Shuffled", img_new)
cv.waitKey(0)
cv.destroyAllWindows()

In [11]:
class Parts:
    top_neighbour = None
    bottom_neighbour = None
    left_neighbour = None
    right_neighbour = None

    def set_neighbour(self, side, neighbour, percentage):
        if side == "T":
            self.top_neighbour = (neighbour, percentage)
        if side == "B":
            self.bottom_neighbour = (neighbour, percentage)
        if side == "L":
            self.left_neighbour = (neighbour, percentage)
        if side == "R":
            self.right_neighbour = (neighbour, percentage)

    def __init__(self, top, bottom, left, right):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        
    def __str__(self):
        return f'Top: {self.top},\nBottom: {self.bottom},\nLeft: {self.left},\nRight: {self.right}\n'

In [12]:
subparts = divide_and_shuffle(img_new, sub_height, sub_width, False)
visited = list()

def part_details(subparts):
    part_info = list()
    for i in subparts:
        temp = Parts(i[0:1, 0:], i[-1:, 0:], i[0:, 0:1], i[0:, -1:])
        part_info.append(temp)
        
    return part_info

In [13]:
def compare_edges(part_a, part_b):
    count = 0
    for a in range(length:=len(part_a)):
        if part_a[a] == part_b[a]:
            count += 1
        else:
            continue

    return round((count / length) * 100, 2)

#Debug
# test = part_details(subparts)
# print(compare_edges(test[0].left, test[1].right))

In [14]:
#Horizontal neighbour check
def compare_hor(part_a, subparts):
    max_percentage = 0
    max_match = [part_a, ""]
    
    #Matching right of part_a with left of all
    curr_percentage = 0
    for x in subparts:
        curr_percentage = compare_edges(part_a.right, x.left)
        if curr_percentage > max_percentage:
            max_match[0] = x
            max_match[1] = "R"
            max_percentage = curr_percentage
    #Matching left of part_a with right of all
    curr_percentage = 0
    for x in subparts:
        curr_percentage = compare_edges(part_a.left, x.right)
        if curr_percentage > max_percentage:
            max_match[0] = x
            max_match[1] = "L"
            max_percentage = curr_percentage

    if max_match[1] == "L":
        part_a.set_neighbour("L", max_match[0], max_percentage)
    else:
        part_a.set_neighbour("R", max_match[0], max_percentage)

    return


In [19]:
import copy
part_info = part_details(subparts)
visited.clear()
for x in range(len(part_info)):
    if x in visited:
        continue
    else:
        visited.append(x)
        one = copy.deepcopy(part_info)
        one.pop(x)
        compare_hor(part_info[x], one)
    

In [20]:
for x in part_info:
    if x.right_neighbour is not None:
        

None
(<__main__.Parts object at 0x7f8e34111900>, 99.57)
(<__main__.Parts object at 0x7f8e34111270>, 99.14)
None
None
(<__main__.Parts object at 0x7f8e341133a0>, 99.14)
None
(<__main__.Parts object at 0x7f8e34113f40>, 99.57)
(<__main__.Parts object at 0x7f8e141e3f40>, 99.57)
None
None
(<__main__.Parts object at 0x7f8e34112e60>, 98.71)
(<__main__.Parts object at 0x7f8e34112fb0>, 99.57)
None
(<__main__.Parts object at 0x7f8e34112e30>, 98.71)
None


In [ ]:
img_gawd=cv.hconcat([p6,p0])
cv.imshow("Pratyush", img_gawd)
cv.waitKey(0)
cv.destroyAllWindows()
        